In [ ]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

## Non-std libs
import matplotlib.pyplot as plt
import pandas as pd

## Local libs
from scn_rrd import rrd_utils, rrd_meta_utils

In [ ]:
locations = [
    'FCS',
    'sps-garfield',
    'occ-oromo',
    'northlake',
]
portGroups = [
    'epc-backhaul-interface',
    'backhaul-interface',
]

meta = rrd_meta_utils.read_meta()
meta = meta[
    ( meta['location'].isin(locations) ) &
    ( meta['port_group_name'].isin(portGroups) )
]

In [ ]:
deviceHostname_to_df = {
    row['hostname']:
    rrd_utils.read_rrd(row['hostname'], row['port_rrd_filename'], '-6month')
    for (_, row) in meta.iterrows()
}

In [ ]:
(fig, ax) = plt.subplots()
for (deviceHostname, df) in deviceHostname_to_df.items():
    inbound_gigabytes = df['INOCTETS'] / pow(2, 30)
    outbound_gigabytes = df['OUTOCTETS'] / pow(2, 30)
    ax.plot(df['time'], inbound_gigabytes, label=f'{deviceHostname} inbound')
    ax.plot(df['time'], outbound_gigabytes, label=f'{deviceHostname} outbound')
ax.set_title('Uplink data rate')
ax.set_ylabel('Gigabyte/sec')
ax.legend()

In [ ]:
fig, axez = plt.subplots(nrows=len(deviceHostname_to_df), figsize=(10,20))
for (ax, (deviceHostname, df)) in zip(axez, deviceHostname_to_df.items()):
    df_monthly = df[['time', 'INOCTETS', 'OUTOCTETS']] \
        .groupby(by=pd.Grouper(key='time', freq='M')) \
        .sum() \
        / pow(2, 30)
    df_monthly.index = df_monthly.index.to_period('M')
    df_monthly.plot(
        ax=ax,
        kind='bar',
        title=deviceHostname,
        xlabel='',
        ylabel='Gigabyte',
    )
fig.suptitle('Monthly data rate')